In [1]:
import gdspy
import qnldraw as qd
import qnldraw.library as qlib
from qnldraw import Chip
import qnldraw.shapes as shapes
from qnldraw import components
import qnldraw.paths as paths
import numpy as np
import qnldraw.junction as qj

In [2]:

class DolanBridgeJunctionArray(components.Component):
    __draw__ = True
    
    def draw(self, n, overlap, spacing, undercut):
        jx, jy = overlap
        ox, oy = undercut
        
        positions = (np.arange(n) - (n-1)/2)*(jy+spacing)
        
        for y in positions:
            junction_rect = shapes.Rectangle(jx, jy).translate(0, y)
            self.add(junction_rect)
        overlay = shapes.Rectangle(ox,oy, layer = 1).translate(-extra_undercut, 0)
        self.add(overlay)
        nodes = {
            'wire1': np.array((0, positions[-1] + 0.5*jy)),
            'wire2': np.array((0, positions[0] - 0.5*jy))
        }
        return nodes
    

In [13]:
if __name__ == "__main__":
    lib = gdspy.GdsLibrary()
    chip = qd.Chip(size = (2000,1000))

#     with open('parameters.yaml', 'r') as f:
#         params = qd.Params(yaml.load(f, yaml.CSafeLoader))

    #These should probably be put in parameters.yaml
    n =  10
    overlap = [2,2] # lx, ly, junction size
    spacing = 0.2
    extra_undercut = 0.5
    undercut = [overlap[0]+0.1+extra_undercut, n*(overlap[1]+spacing)+0.1]
    junction_size = [0.1, 0.1]
    

    dose_layers = [1, 2]
    evap_layers = [4, 5]
    params = {
        'wire': {'width': 0.17, 'taper_length' :0.5},
        'junction':
            {'lengths': 
                {'length': 1.5,
                'horizontal_extra': 0.2,
                'vertical_extra': 1.75},
            'contact': 
                 {'narrow':
                      {'length': 0.1, # 0.150
                      'width_multiplier': 0.3333333333333333 },#multiplier
                  'wide':
                      {'length': 0.1,
                      'width_multiplier': 0.6666666666666667 }},#Multiplier
            'undercut': 
                 {'layer': 2, 'multiplier': 0.6, 'length': 1.5, 'width_multiplier': 2}
            },
        
        'evaporation':
            {'layers': [4, 5],
             'source': {'distance': 390000.0},
             'resist': {'residual_undercut': 0.17, 'upper_photoresist_thickness': 0.18, 'lower_photoresist_thickness': 0.72},
             'angles': {'theta': 45, 'phi': ['-y', '+x']}
            }
        }
    jparams = params['junction']
#     print (jparams['contact']['narrow']['width_multiplier'])
    jparams['contact']['narrow']['width'] = jparams['contact']['narrow']['width_multiplier'] * junction_size[0]
    jparams['contact']['narrow']['width'] = jparams['contact']['wide']['width_multiplier'] * junction_size[0]

    junction = qj.ManhattanJunction(junction_size, wire = params['wire'],**jparams)
    junction_array = DolanBridgeJunctionArray(n, overlap, spacing, undercut)
    j1 = junction.place((-10, 0), node='origin', rotation=0)
    chip.add_component(junction_array, cid='JJArray', layers=dose_layers)
    chip.add_component(j1, cid='JJ', layers=dose_layers)

    ## Simulate Evaporation
    polys = junction_array.get_polygons(by_spec=True)
    highdose = gdspy.PolygonSet(polys[(dose_layers[0], 0)])
    lowdose = gdspy.PolygonSet(polys[(dose_layers[1], 0)])

    evaporated = qj.simulate_evaporation(
        lowdose, highdose, **params['evaporation']
    )

    for i, (layer, evap) in enumerate(zip(evap_layers, evaporated)):
        chip.add_component(evap, f'evap_{i}', layers=layer)
        
    mask = chip.render(name = 'fluxonium', draw_border = True)
#     fig = qd.plot(mask[0], figsize=(20,20), axis=True, grid=False)
    
    cells = chip.render(name='jj_array', draw_border=False)
    lib.write_gds('jj_array_test.gds', cells=cells)